Copyright (c) 2023, Mikołaj Tadeusz Żychowicz (MTZ). All rights reserved.

SlothPy is released under the GNU General Public License v3 (GPLv3). For details, see the [LICENSE](https://github.com/MTZ-dev/slothpy/blob/main/LICENSE.txt) file.

# SlothPy Tutorial
<img src="https://raw.githubusercontent.com/MTZ-dev/slothpy/main/doc/source/_static/slothpy_3.png" width="300">

I designed SlothPy to be used in interactive Jupyter notebooks and stand-alone scripts, which users can create and adjust to their needs, seemingly integrating into custom data pipelines. Remember, especially while working on Windows, to include into your programs `if __name__ == "__main__"` block for executing instructions after importing modules. You can follow the notebook or copy its contents to .py files and execute them as you prefer.

# HDF5 Integration in SlothPy

SlothPy extensively utilizes the HDF5 file format—a versatile and portable binary format optimized for fast I/O operations and handling large datasets. The primary file extension used by SlothPy, `.slt`, is essentially an HDF5 file. This means it can be opened and manipulated with standard HDF5 tools like HDFView and HDFCompass or accessed programmatically via the `h5py` Python interface. Now, let's begin by importing SlothPy along with other essential packages for this tutorial.

In [1]:
import slothpy as slt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Make sure you have access to the libraries on your machine.

ModuleNotFoundError: No module named 'primme'


# Creating a Compound Object in SlothPy

To start using SlothPy, it's essential to create an instance of the `Compound` class, which is a core element of SlothPy. The `Compound` instance is directly linked to a `.slt` file on your disk and provides access to all the methods serving as a user interface/API. You can create a `Compound` object from the output of quantum chemistry software, work with an existing `.slt` file, or append new data to it. Those operations are handled by the Compound creation methods (see [documentation](https://slothpy.org/creation_methods)). In this tutorial, we'll utilize a `.rassi.h5` file from a MOLCAS calculation, available for download [here](https://drive.google.com/file/d/1obU_bK9sc2hdy3vFXjLxtTUX7CBzl-wf/view) or on the [website](https://slothpy.org/how_to_start) (you can try to use your file if you already have access to one). To create a `Compound` from this file, we use the [`slt.compound_from_molcas()`](https://slothpy.org/creation_methods#slothpy.compound_from_molcas) method. It's important to always refer to the [documentation](https://slothpy.org/reference_manual) for details on the methods we use. In Jupyter Notebooks, a quick way to access method docstring is to press `Shift` + `Tab` in parenthesis after typing the method name. You need to provide appropriate paths and names for the files. (you can click the underscored links)

In [ ]:
NdCo = slt.compound_from_molcas(".", "Nd_tutorial", "bas3", ".", "YbCN_DPPMO2_small_bas2")
# Provide paths to the locations of your .rassi.h5 file and where you want to store the resulting .slt file
# replacing ".". You can also adjust the names.

We named the instance `NdCo`, and now it's directly linked to the new file `Nd_tutorial.slt`. After the creation, you can check what is inside your file using the print() method or directly in Notebook like this:

In [ ]:
NdCo

You should see the list of HDF5 groups (with `bas3` among them) and data sets contained in the file, together with their description attributes. This is the way that SlothPy will save all your future results. Having already .slt file on your disk, you can add to it more ab initio results (just use the path and name of an existing file) or access it at a later point using the [`slt.compound_from_slt() method`](https://slothpy.org/creation_methods#slothpy.compound_from_slt). You can also create many instances and files for different compounds or store them all in one file - the decision is up to you. More advanced use cases in actual workflows are demonstrated in Tutorial 2. (If you are interested in adding a new file format for loading energies, spin, and angular momenta data from your favorite quantum-chemistry package, please get in touch with me directly: mikolaj.zychowicz@uj.edu.pl or create an [issue](https://github.com/MTZ-dev/slothpy/issues).)

In [2]:
NdCo = slt.compound_from_slt(".", "Nd_tutorial")
# If you have created "Nd_tutorial.slt" on your disk, only use this line.

## Utilizing SlothPy's  Methods

All available methods are accessed through an instance of the Compound class that constitutes the user interface and API documented in the [Reference Manual](https://slothpy.org/reference_manual). Let us start by computing molar powder-averaged magnetisation as a function of temperature and magnetic field strength `M(T,H)` for our Nd-based compound. We must create one-dimensional lists or arrays of temperature and field values at which the magnetization will be computed. It's convenient to use the numpy linspace method here for fields between 0 and 7 T (50 points) and temperatures from 1 to 10 K with 1 K step.

In [3]:
fields_mth = np.linspace(0.0001, 7, 10000)
temperatures_mth = np.linspace(1, 10, 300)

After that, we should reference the NdCo object. Use `NdCo. + Tab` here in the notebook or autocompletion in our IDE to see the list of all available functions. The method for computing magnetisation as a function of field and temperature is called [`calculate_magnetisation()`](https://slothpy.org/compound#slothpy.Compound.calculate_magnetisation). We need to provide the name of a group from which magnetisation will be calculated along with fields, option for integration grid, temperatures and states cutoff for Zeeman Hamiltonian diagonalization.

In [4]:
%%time
slt.turn_on_monitor()
mth = NdCo.calculate_magnetisation("bas3", fields_mth, 4, temperatures_mth, 14)

Initialization...
Calculate Magnetisation started: 2024-01-22 01:00:53
Progress: |--------------------------------------------------| 0.00% | Time: 0.0 s |
CPU Usage: 3.8% | Memory Usage: 2.7% | Processes: 128 | Threads per Process: 1 Progress: |--------------------------------------------------| 0.00% | Time: 0.2 s |
Progress: |--------------------------------------------------| 0.00% | Time: 0.4 s |
Progress: |--------------------------------------------------| 0.00% | Time: 0.6 s |
Progress: |--------------------------------------------------| 0.00% | Time: 0.8 s |
Progress: |--------------------------------------------------| 0.00% | Time: 1.0 s |
Progress: |#-------------------------------------------------| 2.64% | Time: 1.2 s |
Progress: |#####---------------------------------------------| 10.19% | Time: 1.4 s |
Progress: |########------------------------------------------| 17.88% | Time: 1.6 s |
Progress: |############--------------------------------------| 25.02% | Time: 1.9 s

Here, we calculate powder-averaged (using Lebedev-Laikov integration grid on the hemisphere) magnetisation for the Nd-based compound in the magnetic field range from 1 Oe to 7 T (50 points) and for ten temperature values from 1 K to 10 K. We include in the Zeeman Hamiltonian only 10 Spin-Orbit states from the ground multiplet $ ^{4}I_{9/2} $ of Nd(III). The calculation should finish immediately due to the very low amount of SO states included.

The result is a numpy NdArray (10, 50) with the structure [temperatures, fields] returned from the function call to the mth variable. It is ready for you to process it using Python as you want. Remember to always check the output format of methods in the [Reference Manual](https://slothpy.org/reference_manual) (or using your editor/IDE) - `Returns` section of each function in the documentation.

In [ ]:
mth

# Data Manipulation and Visualisation
SlothPy seamlessly interfaces with the broader toolchain and Python scientific ecosystem by returning data in numpy array format. This compatibility allows you to leverage the full range of Python's data manipulation, exporting, and visualization tools in your scripts using your favorite libraries. As an illustrative example, let's plot the data contained in mth variable as a magnetic field function M(H) iterating over different temperatures using matplotlib:

In [ ]:
for mh in mth:
    plt.plot(fields_mth, mh)
plt.show()

If you need to export the data to use it elsewhere we can simply save it to a .csv file on the disk to be accessible for you or your other programs directly:

In [ ]:
np.savetxt('my_mth_array.csv', mth.T, delimiter=',')
# Here we transpose to have the data for each temperature value in columns

We can make it even better and create a well-formated data frame using pandas library for the data exporting:

In [ ]:
import os

fields_oe = fields_mth * 10000
magnetisation_df = pd.DataFrame({'Fields (Oe)': fields_oe})

for i, mh in enumerate(mth):
    magnetisation_df[f"{temperatures_mth[i]} K"] = mh

magnetisation_df.to_csv(os.path.join(".", "magnetisation.csv"))

As you can see, the possibilities are endless.

Nevertheless, SlothPy offers you its own organized data storage inside the core .slt files along with a series of built-in methods to visualize it and proceed with multi-step complex computations. To demonstrate it, we will now run the above calculation once again, but this time we will use a better, denser grid and include more SO-states. It will take a little more time. Additionally, we will save the results to our Nd_tutorial.slt file using `slt` keyword. (confront the [documentation](https://slothpy.org/compound#slothpy.Compound.calculate_magnetisation) for the comprehensive description of all the options). 

In [ ]:
%%time
mth = NdCo.calculate_magnetisation("bas3", fields_mth, 6, temperatures_mth, 14, slt="bas32")

If you invoke the representation of the Nd_tutorial.slt file once again, you can see that a new group "bas3_magnetisation" was created which contains datasets for magnetisation (bas3_mth), magnetic fields (bas3_fields), and temperatures (bas3_temperatures). (all with description attributes)

In [ ]:
NdCo

SlothPy provides an array-like interface for reading and writing data from and to .slt files. To access a particular dataset and read it in as a numpy array, you need to provide the name of a group followed by the dataset's name. As an example, let us read the magnetisation to another variable mth_custom_read together with field values:

In [ ]:
mth_read = NdCo["bas3_magnetisation", "bas3_mth"]
field_values_read = NdCo["bas3_magnetisation", "bas3_fields"]

Here, we provide a <u>full</u> group and dataset name (with suffixes) to access the data! Now, once again, we can do whatever we want with the arrays. Let us confirm that they indeed represent the same magnetisation as before by plotting them:

In [ ]:
for mh in mth_read:
    plt.plot(field_values_read, mh)
plt.show()

Since we have our data saved in the .slt file, we can plot it using the build-in function (available for various methods and having plenty of customizable parameters - see all the methods starting with "plot" in the [documentation](https://slothpy.org/reference_manual)).

In [7]:
NdCo.plot_magnetisation("bas32")

Figure saved to /home/mikolaj/Downloads/telomelaza.png with DPI=600


As you noticed, when using the built-in functions, you do not need to (actually, you can't) provide a name of the group with a suffix, like "_magnetisation"; just use the name you gave for the `slt` parameter of the method and the program will handle the rest for you.

If you want to integrate the .slt file in your workflow (e.g., adding experimental data), you can even create your custom groups with datasets (in the form of numpy NdArrays) in the file and use them in your programs and scripts simply by using a `Compound` class instance connected to the file - NdCo[...]:

In [ ]:
one_to_ten = np.linspace(1, 10, 10)
NdCo["my_custom_group", "one_to_ten_dataset", "My description of the dataset",
     "My description of the whole group"] = one_to_ten
NdCo

The last two strings, setting a description of the dataset and group, are optional and ordered in such a way as to make it possible to add datasets to the same group later without repeating its description. All the values you can store must be in an ArrayLike form (can be converted to the numpy NdArray). As an example, we add to our new group a new dataset with a description, a new dataset without a description, and a stand-alone dataset without any group. (the data must be at least one-dimensional)

In [ ]:
NdCo["my_custom_group", "dataset_with_description", "Identity matrix"] = [[1,0,0],[0,1,0],[0,0,1]]
NdCo["my_custom_group", "123_dataset"] = [1,2,3]
NdCo["my_dataset_without_a_group"] = [1]
NdCo

Suppose you now re-run the previous calculation. In that case, you should see a SlothPyError due to the already existing name of the group (SlothPy prevents you from accidentally overwriting the data). The library has extensive custom error handling, eliminating tedious, long tracebacks and trying to inform users directly about all sorts of potential problems, hopefully making the user experience less painful.

In [ ]:
%%time
mth = NdCo.calculate_magnetisation("bas3", fields_mth, 6, temperatures_mth, 32, slt="bas3")

Finally, we can use [delete_group_dataset()](https://slothpy.org/compound#slothpy.Compound.delete_group_dataset) to manually remove datases/groups from the .slt file.

In [ ]:
NdCo.delete_group_dataset("my_dataset_without_a_group")
NdCo.delete_group_dataset("my_custom_group", "123_dataset") # Only the 123_dataset is removed
NdCo.delete_group_dataset("bas3_magnetisation") # The whole group is removed
NdCo

As a final remark, remember that the .slt files are, in fact, HDF5 files in disguise. That means you can obtain all the above functionalities (and much more) directly using, for example, the h5py Python wrapper in your scripts for .slt file reading and modifications, further incorporating them into your data pipelines.

# Optimizing Computational Performance with SlothPy:
# Parallel Processing and Autotuning
As you should have already noticed (when reading the docstring of [calculate_magnetisation()](https://slothpy.org/compound#slothpy.Compound.calculate_magnetisation) method), SlopthPy provides you full control over the number of CPUs you want to assign to your calculation and the threads to be used for linear algebra libraries. Computationally demanding methods are parallelized using a certain amount of separate Python processes - the number of processes that will be used is (number of CPUs) // (number of threads). Additionally, in the "Note" section of each method's documentation, the user is informed over what quantity the job will be parallelized. In the case of [calculate_magnetisation()](https://slothpy.org/compound#slothpy.Compound.calculate_magnetisation)  method, the work is distributed over different field values (here 50). So, when using, for example, 10 parallel processes, each will compute the magnetisation for 5 values of the magnetic field. As default, SlothPy uses all available logical cores with 1 thread assigned for the linear algebra libraries. For jobs with a very high number of points to be parallelized over, you should benefit from a greater number of processes. On the other hand, with increasing matrix sizes, it is beneficial to use more threads for operations such as diagonalization, dot products, etc. It is not a trivial task to choose optimal settings for very demanding calculations, and that is why we provide, within SlothPy, the autotune module to do it for you. It tests all possible meaningful setups and gives you time estimates for each of them. It takes some time to do this (because it actually truly does a part of the calculations to benchmark them), so it is advised to use it for jobs that will take hours. To demonstrate it with relatively small matrices available in our file (they are at most 364 x 364 with states_cutoff set to 0 - that is how many SO states there are), we will run two examples using all available CPUs on your machine (if you want to leave some for other tasks you should change number_cpu = 0 to your desired number):

In [ ]:
fields_process = np.linspace(0.0001, 7, 60)
fields_threads = np.linspace(0.0001, 7, 3)

In [ ]:
%%time
mth = NdCo.calculate_magnetisation("bas3", fields_process, 6, temperatures_mth, 14, number_cpu = 16, autotune=True)

In [ ]:
%%time
mth = NdCo.calculate_magnetisation("bas3", fields_threads, 6, temperatures_mth, 14, number_cpu = 16, autotune=True)

In the first case, the autotune module should choose (but it depends on your hardware and CPU number) more processes and fewer threads than in the second one, where we parallelize only over 3 field points. Time estimates include only pure calculation steps, and in our tests for a variety of different methods, they should give results within 20-30% maximal error of overall execution time. After the autotuning, you can run the calculation with the chosen setting manually to see how much time it will take compared to our estimate. Can you choose better settings by yourself?

In [ ]:
num_of_cpu = 16 #fill here the number you were autotuning for
num_of_threads = 2 #fill here the number of threads chosen by the autotune module (for fields_process and _threads)

In [ ]:
%%time
mth = NdCo.calculate_magnetisation("bas3", fields_process, 6, temperatures_mth, 14, num_of_cpu, num_of_threads)

In [ ]:
%%time
mth = NdCo.calculate_magnetisation("bas3", fields_threads, 6, temperatures_mth, 14, num_of_cpu, num_of_threads)

The necessity of the autotune module will become visible for matrices with a number of states over 500-1000 or even 2000+ when calculations with certain settings (how many field values and grids) can take many hours or even days. It also all depends on your hardware e.g. how many possibilities is there to check. For me writing this tutorial now I am testing it on 128 logical cores (64 physical) CPU, so there are many possibilities to choose a number of threads and processes - therefore it is harder and also more time-consuming for the module, but still better than trying manually all the possibilities.

In the following part of the tutorial, we will give a glimpse into more methods presenting their use for our Nd-based compound. Make sure to check the documentation of each presented method - it always contains a comprehensive description of all the used parameters. We begin by calculating magnetic susceptibility - in the form of a product with temperature - (the derivative of magnetisation with respect to the magnetic field) in 1000, 2000, 3000, and 5000 Oe.

In [ ]:
temperatures = np.linspace(1, 300, 300)
fields = [0.1, 0.2, 0.3, 0.5]

In [ ]:
%%time
chitht = NdCo.calculate_susceptibility("bas3", temperatures, fields, number_of_points=2, delta_h=0.0001,
                               states_cutoff=14, number_cpu=0, number_threads=1, T=True, slt="your_name")

In [ ]:
NdCo.plot_susceptibility("your_name")

You can calculate the Helmholtz free energy or internal energy in the applied magnetic field (here instead of averaging we calculate with the field applied in the "z" direction):

In [ ]:
temperatures = np.linspace(1, 300, 300)
fields = np.linspace(0.0001, 10, 100)

In [ ]:
%%time
eth = NdCo.calculate_energy("bas3", fields, [[0., 0., 1., 1.]], temperatures, "helmholtz", 14, number_cpu=0, 
                                  number_threads=1, slt="your_name")

In [ ]:
NdCo.plot_energy("your_name", "helmholtz")

You can calculate the Zeeman splitting of the ground $ ^{4}I_{9/2} $ multiplet for various directions (here x, y, z) or take powder-average:

In [ ]:
%%time
zeeman = NdCo.calculate_zeeman_splitting("bas3", 8, fields, [[1,0,0],[0,1,0],[0,0,1]], states_cutoff=14,
                                number_cpu=0, number_threads=1, average=False, slt="your_namee")

In [ ]:
NdCo.plot_zeeman("your_namee")

In [ ]:
g, axes = NdCo.calculate_g_tensor_and_axes_doublet("bas3", [0], slt="axes")

SlothPy allows you to calculate directional data of the above quantities in the form of 3D plots over spherical angles. This type of calculation is pretty demanding and heavy on memory usage. See the Note sections for the following methods to see examples of memory estimation that is needed to handle big resulting and intermediate arrays!!! (example: for a calculation with 100 field values 1-10 T, 300 temperatures 1-300 K,
and spherical_grid = 60, the resulting array will take 3 * 100 * 300 * 2 * 60 * 60 * 8 bytes = 5.184 GB)

In [ ]:
g, axes = NdCo.calculate_g_tensor_and_axes_doublet("bas3", [0])
axes = axes[0].T
axes

In [ ]:
%%time
r = NdCo.calculate_energy_3d("bas3", fields, "mesh", 35, temperatures, "helmholtz", 14, slt="your_name", number_cpu=16, rotation=axes)

In [ ]:
%%time
r = NdCo.calculate_magnetisation_3d("bas3", fields, "mesh", 35, temperatures, 14, slt="your_name", number_cpu=16, rotation=axes)

In [ ]:
%%time
r = NdCo.calculate_susceptibility_3d("bas3", temperatures, fields, "mesh", 35, 2, states_cutoff=14,  number_cpu=16, slt="your_name", rotation=axes)
# Carefull! This is the most demanding calculation it has to perform (2*number_of_points+1) times the 
# magnetisation calculation for numerical differentiation using finite difference method with the custom stencil.

At this point, you may need to adjust (lower or raise) states_cutoff-s, density of spherical grid, or number of field values depending on your hardware (CPU and memory) capability. Also, the autotune module should always choose 1 thread due to the large number of the point we parallelize over (number of fields) * 2 * spherical_grid**2, since your matrices are small and can have at most 364 x 364 size it cannot prioritize multithreading for them.  

After you succeed we can plot the results for particular temperatures and field values with the plot_3d method which supports energy, magnetisation, and susceptibility plotting (see the datatype keyword):

In [ ]:
NdCo.plot_3d("your_name", "helmholtz_energy", 10, 30) 
# Options: "helmholtz_energy", "internal_energy", "magnetisation", "chi", "chit"

Anyway, probably the best method to examine this type of data is to scan it using an interactive 3D plot where you can change fields and temperatures using sliders and see changes in real-time!

In [ ]:
NdCo.interactive_plot_3d("your_name", "chit", add_g_tensor_axes=True, axes_group="axes", axes_scale_factor=1.5,
                        doublet_number=0, rotation=axes)
# Options: "helmholtz_energy", "internal_energy", "magnetisation", "chi", "chit"

After you manually find all the instersting "phase" transitions you can even prepare .gif animations varying temperature or field strenght while keepeing the other constant.

In [ ]:
NdCo.animate_3d("your_name", "helmholtz_energy", "temperature", "animation", i_start=0, i_end=30, i_constant=2,
               fps=6, dpi=200)

The animation should be saved to your current directory.

We will end this basic tutorial just by mentioning other methods. More advanced combinations resulting in e.g. studies on relaxation dynamics for molecular magnets will be addressed in more specialized tutorials.

In order to calculate pseudo-g-tensors for doublet states within the ground $ ^{4}I_{9/2} $ multiplet (10 states - 5 doublets) you can run the:

In [ ]:
g_tensors, magnetic_axes = NdCo.calculate_g_tensor_and_axes_doublet("bas3", [0,1,2,3,4]) 

In [ ]:
g_tensors, magnetic_axes

The first entry of the tuple contains g-tensor components x, y, and z for the doublets and magnetic_axes are rotation matrices from the initial coordinate system to the main magnetic axes of each doublet (Coordinates of the main axes X, Y, and Z in the initial x, y, z frame are columns of such matrices). We can use those matrices to rotate angular momenta in almost all the following methods to express the quantities in the reference frame of the chosen doublet's magnetic axes. Note, that you need to use the inverse rotation (transpose of an orthogonal matrix) to express everything in the new reference frame! (see also Exporting Module for an easy way to add the main magnetic axes to your .mol2/.xyz files with molecular coordinates)

In [ ]:
your_rotation = magnetic_axes[0].T # Inverse rotation for the ground doublet
your_rotation

As an example, we calculate the susceptibility (Van-Vleck) tensor using true numerical differentiation in the reference frame of the ground doublet state:

In [ ]:
temperatures = linspace(1, 300, 300)
fields = [0.1, 0.2]

In [ ]:
NdCo.calculate_chit_tensorht("bas3", temperatures, fields, number_of_points=3, delta_h=0.0001, states_cutoff=364,
                             number_cpu=0, number_threads=1, rotation=your_rotation)

Now we will do a little demonstration considering our 10-state ground manifold. Firstly, let us take a look at SOC energies of the states:

In [ ]:
NdCo.soc_energies_cm_1("bas3", 10)

They come in doublets as expected for the Krammers ion. We can then obtain Crystal-Fields-Parameters for S = 9/2 pseudo-spin (in the form of J - total angular momneta) for the SOC matrix taking as "z" quantization axis the main magnetic axis of a ground doublet (use your_rotation). Note that the order of CFPs cannot exceed 2S here and because SOC Hamiltonian (without magnetic fields) is time even operator we only need even orders. We will use real parameters.

In [ ]:
NdCo.soc_crystal_field_parameters("bas3", 0, 9, order=9, pseudo_kind = "magnetic", 
                                  even_order=True, rotation=your_rotation, slt="your_name")

For the comparison let us get the initial SOC matrix in the same pseudo-spin Jz basis:

In [ ]:
soc_matrix = NdCo.soc_zeem_in_z_angular_magnetic_momentum_basis("bas3", 0, 9, "soc", "magnetic",
                                                                rotation=your_rotation)
soc_matrix

Verify it by computing its eigenvalues which should be the same as SOC energies from before.

In [ ]:
from numpy.linalg import eigvalsh
energies = eigvalsh(soc_matrix) * 219474.6 # Convert it from Hartree to cm-1
energies

We can rebuild the whole matrix from the saved CFP (ITO) parameters and verify that it is the same as soc_matrix (giving the same eigenvalues):

In [ ]:
soc_matrix_from_cfp = NdCo.matrix_from_ito("your_name_soc_ito_decomposition", False)

energies = eigvalsh(soc_matrix_from_cfp) * 219474.6 # Convert it from Hartree to cm-1
energies

In [ ]:
soc_matrix - soc_matrix_from_cfp

If you need numerical precision for the matrix recreation, you should use odd orders of ITOs (Irreducible Tensor Operators) as well:

In [ ]:
NdCo.soc_crystal_field_parameters("bas3", 0, 9, order=9, pseudo_kind = "magnetic", 
                                  even_order=False, rotation=your_rotation, slt="your_name_odd")

In [ ]:
soc_matrix_from_cfp_odd = NdCo.matrix_from_ito("your_name_odd_soc_ito_decomposition", False)

energies = eigvalsh(soc_matrix_from_cfp) * 219474.6 # Convert it from Hartree to cm-1
energies

In [ ]:
soc_matrix - soc_matrix_from_cfp_odd

Using all orders is necessary e.g. when dealing with Zeeman Hamiltonians containing interaction with magnetic field on top of the energy of SOC states. We can follow the seam procedure for a full Zeeman matrix originating from interaction with a magnetic field in the "y" direction (2 T):

In [ ]:
NdCo.zeeman_matrix_ito_decpomosition("bas3", 0, 9, 2, [0., 1., 0.], order=9, pseudo_kind = "magnetic", 
                                     rotation=your_rotation, slt="your_name")

In [ ]:
zeeman_matrix = NdCo.soc_zeem_in_z_angular_magnetic_momentum_basis("bas3", 0, 9, "zeeman", "magnetic", field=2.,
                                                                   orientation= [0., 1., 0.],
                                                                   rotation=your_rotation)
zeeman_matrix

In [ ]:
zeeman_matrix_from_cfp = NdCo.matrix_from_ito("your_name_zeeman_ito_decomposition", False)

energies = eigvalsh(zeeman_matrix_from_cfp) * 219474.6 # Convert it from Hartree to cm-1
energies # Zeeman splitting

In [ ]:
energies = eigvalsh(zeeman_matrix) * 219474.6 # Convert it from Hartree to cm-1
energies # Zeeman splitting

In [ ]:
zeeman_matrix - zeeman_matrix_from_cfp

Feel free to experiment with two available pseudo-spin bases (total angular or magnetic).

We will end this short introduction to SlothPy software by showing how to obtain a % decomposition in the pseudo-spin basis of a SOC/Zeeman matrix.

In [ ]:
NdCo.matrix_decomposition_in_z_pseudo_spin_basis("bas3", "soc", "magnetic", 0, 9, rotation=your_rotation)

In [ ]:
NdCo.matrix_decomposition_in_z_pseudo_spin_basis("bas3", "zeeman", "magnetic", 0, 9, rotation=your_rotation,
                                                orientation=[0., 1., 0.], field=2.)

Here, the resulting matrix contains % weights of pseudo-spin states (columns - from -S to S) in each state from the list (rows - here 10 states from 0 to 9) where pseudo-spin number S = number_of_states / 2 - 1/2 = 10/2 - 1/2 = 9/2.

To explore more methods check the Reference Manual and more specialized, upcoming tutorials.

MTZ